***S metaheuristike (single solution)***


Zadatak: ***UFLP*** \\
*I* korisnika \\
*J* resursa \\
$C_{ij}$ - cena pridruzivanja korisnika *i* resursu *j* \\
$f_j$ - cena uspostavljanja resursa *j* \\


---


Kodiranje resenja: [True, False, True, False, True] \\
Okolina resenja:   [True, False, True, True, True]

**1. Lokalna pretraga**

In [2]:
import random

In [1]:
# Da li je resenje dopustivo (nisu sve vrednosti jednako False)?
def isFeasible(solution):
    for resource in solution:
        if resource: # bar jedan je jednak True
            return True

    return False

In [3]:
def initialize(numResources, p):
    solution = []

    for i in range(numResources):
        # ako je random.random() < p, u solution se dodaje True
        solution.append(random.random() < p)

    # ako nije dopustivo stavljamo random bit na True
    if not isFeasible(solution):
        solution[random.randrange(numResources)] = True

    return solution

In [12]:
def calcSolutionValue(solution, cost, fixedCost):
    value = 0.0
    
    # slozenost od len() je O(1)
    numUsers = len(cost)
    numResources = len(fixedCost)

    used = [False for _ in range(numResources)]

    # prolazimo kroz sve korisnike
    for i in range(numUsers):
        minCost = float('inf')
        resourceUsed = -1

        # trazimo najblizi resurs koji je uspostavljen za i-tog korisnika
        for j in range(numResources):
            if solution[j] == True and cost[i][j] < minCost:
                minCost = cost[i][j]
                resourceUsed = j

        value += minCost
        used[resourceUsed] = True

    for j in range(numResources):
        # ako je resurs iskoriscen
        if used[j]:
            # dodajemo cenu uspostavljanja tog resursa
            value += fixedCost[j]

    # cuvamo one koje smo iskoristili
    solution = used

    return value

In [6]:
def invert(solution):
    chosen = random.randrange(len(solution))
    solution[chosen] = not solution[chosen]

    # provera da li je novo resenje dopustivo
    if isFeasible(solution):
        return chosen

    # inace, vracamo na staro i vracamo -1
    solution[chosen] = not solution[chosen]
    return -1

In [7]:
def restore(solution, j):
    solution[j] = not solution[j]

In [14]:
def localSearch(cost, fixedCost, iters):
    solution = initialize(numResources=len(fixedCost), p=0.25)
    # trenutna vrednost
    currValue = calcSolutionValue(solution, cost, fixedCost)
    # najbolja vrednost
    bestValue = currValue

    for i in range(iters): # kriterijum zaustavljanja
        # promena trenutnog resenja (invertujemo jednu vrednost)
        inverted_index = invert(solution)
        if inverted_index < 0:
            continue

        newValue = calcSolutionValue(solution, cost, fixedCost)

        # ako je nova vrednost bolja (manja) azuriramo trenutnu vrednost
        if newValue < currValue:
            currValue = newValue
            if newValue < bestValue:
                bestValue = newValue
        else:
            # nastavi petlju sa starim solution
            restore(solution, inverted_index)

    return bestValue

In [8]:
def readInput(filename):
    # ucitavamo podatke iz datoteke
    with open(filename, 'r') as f:
        # citamo prvi red u kome se nalaze dva broja
        numUsers, numResources = [int(x) for x in f.readline().split()]
        # citamo naredne redove koji predstavljaju matricu cene
        cost = [[int(j) for j in f.readline().split()] for i in range(numUsers)]
        # citamo poslednji red u kom se nalaze cene uspostavljanja resursa
        fixedCost = [int(x) for x in f.readline().split()]

    return numUsers, numResources, cost, fixedCost

In [13]:
numUsers, numResources, cost, fixedCost = readInput('in.txt')
ls = localSearch(cost, fixedCost, 10000)
print(ls)

34.0


**2. Simulirano kaljenje**

In [15]:
# Simulirano kaljenje daje mogucnost da se pomerimo i u losijem
# smeru kako bismo kasnije mozda dosli do globalnog minimuma
def simulatedAnnealing(cost, fixedCost, iters):
    solution = initialize(numResources=len(fixedCost), p=0.25)
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue

    for i in range(iters): 
        inverted_index = invert(solution)
        if inverted_index < 0:
            continue

        newValue = calcSolutionValue(solution, cost, fixedCost)

        if newValue < currValue:
            currValue = newValue
            if newValue < bestValue:
                bestValue = newValue
        # jedina izmena u odnosu na (1) je ovde
        # sada prihvatamo i losije resenje
        else:
            p = 1.0 / i**0.5
            q = random.uniform(0, 1)
            if p > q:
                currValue = newValue
            else:
                restore(solution, inverted_index)

    return bestValue

In [17]:
sa = simulatedAnnealing(cost, fixedCost, 10000)
print(sa)

34.0


3. Promenljive okoline (dodatno)

In [19]:
def restoreMultiple(solution, inverted):
    for j in inverted:
        solution[j] = not solution[j]

In [23]:
# menjamo k bitova
def getNeighbor(k, solution):
    numResources = len(solution)
    inverted = []

    for i in range(k):
        j = random.randrange(numResources)

        # sprecavamo menjanje j vise puta
        while j in inverted:
            j = random.randrange(numResources)

        solution[j] = not solution[j]
        inverted.append(j)

    if isFeasible(solution):
        return True, inverted

    else:
        restoreMultiple(solution, inverted)
        return False, inverted

In [21]:
# U ovom slucaju imamo vise okolina
# RVNS - redukovana metoda promenljive okoline
def RVNS(cost, fixedCost, iters, kMax):
    solution = initialize(len(fixedCost), 0.25)
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue

    for i in range(iters):
        k = 0

        # prolazimo kroz sve okoline
        while k <= kMax:
            # trazimo suseda u k-toj okolini
            feasible, inverted = getNeighbor(k, solution)
            if not feasible:
                continue

            newValue = calcSolutionValue(solution, cost, fixedCost)
            if newValue < currValue:
                currValue = newValue
                k = 0
                if newValue < bestValue:
                    bestValue = newValue
            else:
                # kao u lokalnoj samo ovde menjamo vise bitova
                restoreMultiple(solution, inverted)
                k += 1

    return bestValue

In [24]:
rvns = RVNS(cost, fixedCost, 10, 3)
print(rvns)

34.0
